In [1]:
import numpy as np
import miniImagenetdata as dataset
import os
import pandas as pd
import tensorflow as tf
from PIL import Image
import glob
from scipy import misc

In [2]:
batch_size = 20
ways = 5
shots = 5
query_size = 15
image_shape = [224, 224, 3]
restore = False
total_epochs = 50
total_training_episodes = 1000
total_val_episodes = 250
total_test_episodes = 250
data_format = 'channels_last'
path = os.getcwd()
data_dir = path + '/miniImagenet/'
csv_dir = path + '/miniImagenet/csv/'

In [3]:
data_path = csv_dir + 'tra.csv'
data_set = pd.read_csv(data_path, sep=',')
labels = data_set.label.unique().tolist()

In [4]:
data = dataset.MiniImagenetData(data_dir = data_dir, image_shape= image_shape, batch_size=batch_size, ways=ways, shots=shots, query_size = query_size)

In [5]:
support_set_x, support_set_y, query_x, query_y = data.get_batch('tra')

In [8]:
support_set_y = 

<tf.Tensor 'one_hot:0' shape=(5,) dtype=float32>

In [6]:
support_set_y = tf.one_hot(support_set_y, 5)

In [7]:
support_set_y[1]

<tf.Tensor 'strided_slice:0' shape=(5, 5, 5) dtype=float32>

In [10]:
query_y[1]

array([4., 2., 0., 0., 0., 0., 3., 4., 0., 0., 3., 1., 3., 2., 4.],
      dtype=float32)

In [78]:
pre = tf.argmax(support_set_y, axis=1)

In [79]:
pre

<tf.Tensor 'ArgMax:0' shape=(20,) dtype=int64>

In [71]:
support_set_x = np.reshape(support_set_x,newshape=[batch_size, ways*shots, image_shape[0], image_shape[1],image_shape[2]])
support_set_y = np.reshape(support_set_y, newshape=[batch_size, ways*shots])

In [72]:
su

(20, 25)

In [58]:
support_set_y = tf.expand_dims(support_set_y, axis=2)

In [59]:
support_set_y = tf.tile(support_set_y, [1, 1, query_size])

In [62]:
query_y = tf.expand_dims(query_y, axis=1)

In [63]:
query_y = tf.tile(query_y, [1, ways*shots, 1])


In [68]:
query_y = tf.reduce_sum(query_y, axis=-1)

In [69]:
query_y

<tf.Tensor 'Sum:0' shape=(20, 25) dtype=float32>

In [65]:
labels = tf.equal(support_set_y, query_y)

In [66]:
labels

<tf.Tensor 'Equal:0' shape=(20, 25, 15) dtype=bool>

In [34]:
support_set_x = tf.reshape(support_set_x, shape=[batch_size*ways*shots, image_shape[0], image_shape[1], image_shape[2]])

In [52]:
support_images_embeddings = []

In [53]:
for img in tf.unstack(support_set_x, axis=1):
    support_images_embeddings.append(img)

In [54]:
support_images_embeddings

[<tf.Tensor 'unstack_2:0' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:1' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:2' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:3' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:4' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:5' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:6' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:7' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:8' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:9' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:10' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:11' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:12' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:13' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'unstack_2:14' shape=(20, 224, 224, 3) dtype=float32>,
 <tf.

In [45]:
support_images_embeddings = tf.reshape(support_images_embeddings, shape=[batch_size,ways*shots, image_shape[0], image_shape[1], image_shape[2]])

In [46]:
support_images_embeddings

<tf.Tensor 'Reshape_2:0' shape=(20, 25, 224, 224, 3) dtype=float32>

In [15]:
p_s = np.random.permutation(support_set_x[1].shape[0])

In [16]:
np.take(support_set_x,p_s,axis=1,out=support_set_x)

array([[[[[125., 122., 113.],
          [134., 128., 116.],
          [141., 132., 117.],
          ...,
          [159., 144.,  43.],
          [194., 173.,  58.],
          [188., 176.,  54.]],

         [[126., 124., 112.],
          [134., 128., 114.],
          [142., 133., 116.],
          ...,
          [178., 154.,  46.],
          [207., 180.,  63.],
          [207., 189.,  61.]],

         [[126., 124., 112.],
          [134., 128., 114.],
          [142., 133., 116.],
          ...,
          [188., 160.,  50.],
          [202., 177.,  61.],
          [203., 184.,  63.]],

         ...,

         [[ 23.,  28.,  22.],
          [ 20.,  15.,  12.],
          [ 10.,   9.,   7.],
          ...,
          [  5.,   5.,   5.],
          [  6.,   6.,   6.],
          [  6.,   6.,   6.]],

         [[ 56.,  62.,  62.],
          [ 68.,  67.,  83.],
          [ 58.,  57.,  73.],
          ...,
          [  5.,   5.,   5.],
          [  6.,   6.,   6.],
          [  6.,   6.,   6.]],



In [17]:
sampled_classes = np.random.choice(labels, 1, replace=False)

In [17]:
sampled_classes

array(['n04275548'], dtype='<U9')

In [10]:
img_folder = data_dir + 'tra_data/n04275548'

In [11]:
files = os.listdir(img_folder)

['95.png',
 '396.png',
 '482.png',
 '217.png',
 '594.png',
 '61.png',
 '513.png',
 '490.png',
 '550.png',
 '356.png',
 '319.png',
 '244.png',
 '187.png',
 '188.png',
 '235.png',
 '353.png',
 '399.png',
 '570.png',
 '401.png',
 '328.png',
 '93.png',
 '151.png',
 '67.png',
 '524.png',
 '78.png',
 '157.png',
 '348.png',
 '215.png',
 '457.png',
 '144.png',
 '499.png',
 '168.png',
 '312.png',
 '24.png',
 '420.png',
 '75.png',
 '179.png',
 '415.png',
 '326.png',
 '495.png',
 '74.png',
 '276.png',
 '405.png',
 '258.png',
 '504.png',
 '425.png',
 '239.png',
 '119.png',
 '77.png',
 '292.png',
 '192.png',
 '439.png',
 '411.png',
 '196.png',
 '19.png',
 '12.png',
 '0.png',
 '13.png',
 '222.png',
 '445.png',
 '229.png',
 '107.png',
 '155.png',
 '230.png',
 '385.png',
 '146.png',
 '351.png',
 '279.png',
 '596.png',
 '288.png',
 '43.png',
 '8.png',
 '538.png',
 '519.png',
 '369.png',
 '48.png',
 '418.png',
 '477.png',
 '173.png',
 '394.png',
 '175.png',
 '221.png',
 '537.png',
 '169.png',
 '417.png'

In [59]:
sample_imgs = np.random.choice(files, 8, replace=False)

In [60]:
sample_imgs

array(['179.png', '155.png', '523.png', '97.png', '177.png', '535.png',
       '566.png', '157.png'], dtype='<U7')

In [61]:
folder = path + '/miniImagenet/tra_data/'

In [63]:
img_floder = folder + 'n04275548/'

In [68]:
files = glob.glob(img_folder + '/*.png')

['/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/95.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/396.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/482.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/217.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/594.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/61.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/513.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/490.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/550.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/356.png',
 '/home/dl-box/PycharmProjects/few_shot_learning/miniImagenet/tra_data/n04275548/319.png',
 

In [62]:
for j, img in enumerate(sample_imgs):
    print(j)
    print(img)
    f = Image.open(folder +'n04275548/'+ img)
    f = np.asarray(f, dtype= np.float32)
    print(f.shape)

0
179.png
(224, 224, 3)
1
155.png
(224, 224, 3)
2
523.png
(224, 224, 3)
3
97.png
(224, 224, 3)
4
177.png
(224, 224, 3)
5
535.png
(224, 224, 3)
6
566.png
(224, 224, 3)
7
157.png
(224, 224, 3)
